# Setup

## Installs

In [1]:
!git clone https://github.com/ostris/ai-toolkit
!mkdir -p /content/dataset

Cloning into 'ai-toolkit'...
remote: Enumerating objects: 3962, done.
remote: Counting objects: 100% (2046/2046), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 3962 (delta 1931), reused 1822 (delta 1802), pack-reused 1916 (from 1)
Receiving objects: 100% (3962/3962), 29.71 MiB | 14.40 MiB/s, done.
Resolving deltas: 100% (3015/3015), done.


In [2]:
!cd ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt


Submodule 'repositories/batch_annotator' (https://github.com/ostris/batch-annotator) registered for path 'repositories/batch_annotator'
Submodule 'repositories/ipadapter' (https://github.com/tencent-ailab/IP-Adapter.git) registered for path 'repositories/ipadapter'
Submodule 'repositories/leco' (https://github.com/p1atdev/LECO) registered for path 'repositories/leco'
Submodule 'repositories/sd-scripts' (https://github.com/kohya-ss/sd-scripts.git) registered for path 'repositories/sd-scripts'
Cloning into '/content/ai-toolkit/repositories/batch_annotator'...
Cloning into '/content/ai-toolkit/repositories/ipadapter'...
Cloning into '/content/ai-toolkit/repositories/leco'...
Cloning into '/content/ai-toolkit/repositories/sd-scripts'...
Submodule path 'repositories/batch_annotator': checked out '420e142f6ad3cc14b3ea0500affc2c6c7e7544bf'
Submodule 'repositories/controlnet' (https://github.com/lllyasviel/ControlNet-v1-1-nightly.git) registered for path 'repositories/batch_annotator/repositor

In [3]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00


## Imports

In [5]:

import getpass
import yaml
import os
import sys
sys.path.append('/content/ai-toolkit')
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
import shutil
from toolkit.job import run_job
from collections import OrderedDict
from google.colab import userdata, drive, files
from PIL import Image
import datasets
import prep_utils


In [ ]:
# del sys.modules['prep_utils']

## Set Huggingface Token

In [8]:
os.environ['HF_TOKEN']= userdata.get('Huggingface')

# Training


## single training run

In [11]:
base_dir = '/content/drive/MyDrive/training_images' # path to all training images in google drive
train_directory = '/content/train_images' # path to move the training images to
results_dir = '/content/drive/My Drive/results' # path to result models/embeddings
subfolders = ['textinv', 'dbooth', 'dlora', 'lora'] # subfolders for creating the folder structure

# adjust the parameters below depending on the training process
training_method = 'lora' # training method in this notebook
training_model = 'flux' # model version in this notebook
training_subject = 'leander' # subject reference to train on

prep_utils.create_folder_structure(results_dir, subfolders)

prep_utils.setup_training_environment(train_directory, results_dir, training_model, training_method, training_subject)


Mounted at /content/drive
Root directory already exists: /content/drive/My Drive/results
Subfolder already exists: /content/drive/My Drive/results/textinv
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/SDv1.5
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/flux
Subfolder already exists: /content/drive/My Drive/results/dbooth
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/SDv1.5
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/flux
Subfolder already exists: /content/drive/My Drive/results/dlora
Sub-subfolder already exists: /content/drive/My Drive/results/dlora/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/dlora/SDv1.5
Sub-subfolder already exists: /content/drive/My Drive/results/dlora/flux
Subfolder already exists: /content/drive/My 

In [12]:
images = prep_utils.load_images(base_dir, training_subject)


In [13]:
prep_utils.move_images(training_subject, images, train_directory)

Saved /content/train_images/leander_1.jpg
Saved /content/train_images/leander_2.jpg
Saved /content/train_images/leander_3.jpg
Saved /content/train_images/leander_4.jpg
Saved /content/train_images/leander_5.jpg
Saved /content/train_images/leander_6.jpg
Saved /content/train_images/leander_7.jpg
Saved /content/train_images/leander_8.jpg
Saved /content/train_images/leander_9.jpg
Saved /content/train_images/leander_10.jpg


In [22]:
!echo $TRIGGER_WORD
!echo $MODEL_NAME
!echo $GENDER

ohwx
black-forest-labs/FLUX.1-dev
man


# training loop

In [6]:
base_dir = '/content/drive/MyDrive/training_images' # path to all training images in google drive
train_directory = '/content/train_images' # path to move the training images to
results_dir = '/content/drive/My Drive/results' # path to result models/embeddings
subfolders = ['textinv', 'dbooth', 'dlora', 'lora'] # subfolders for creating the folder structure

# adjust the parameters below depending on the training process
training_method = 'lora' # training method in this notebook
training_model = 'flux' # model version in this notebook
subject_list = ['hannah', 'celine', 'timh', 'timk', 'leander', 'patrick', 'christoph', 'jannik', 'marco', 'nils'] # only needed for complete training loop,

prep_utils.create_folder_structure(results_dir, subfolders)

Mounted at /content/drive
Root directory already exists: /content/drive/My Drive/results
Subfolder already exists: /content/drive/My Drive/results/textinv
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/SDv1.5
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/flux
Subfolder already exists: /content/drive/My Drive/results/dbooth
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/SDv1.5
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/flux
Subfolder already exists: /content/drive/My Drive/results/dlora
Sub-subfolder already exists: /content/drive/My Drive/results/dlora/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/dlora/SDv1.5
Sub-subfolder already exists: /content/drive/My Drive/results/dlora/flux
Subfolder already exists: /content/drive/My 

In [7]:
jobs = []
for subject in subject_list:
    # name = f"flux_lora_{subject}"

    # model_name = os.environ.get('MODEL_NAME')
    prep_utils.setup_training_environment(train_directory, results_dir, training_model, training_method, subject)
    #TODO: NEED TO SET ENVIRON VAR CORRECTLY BEFORE CREATING THE JOB CONFIG FILES?! ->run setup here?

    job_config = prep_utils.create_job_config(name=subject,
                                              max_steps=3000,
                                              save_steps=200,
                                              lr=1e-4,
                                              train_directory=train_directory,
                                              valid_steps=100
                                              )
    jobs.append(job_config)

os.environ.pop('TRIGGER_WORD', None)
os.environ.pop('GENDER', None)
os.environ.pop('OUTPUT_DIR', None)

'/content/drive/My Drive/results/lora/flux/nils'

In [39]:
# Save each job configuration to a file
# for i, job in enumerate(jobs):
#     with open(f"{job['config']['name']}.yaml", "w") as f:
#         yaml.dump(job, f, default_flow_style=False)


In [9]:
# Run each job sequentially
for i, job in enumerate(jobs):
    print(f"Running job {i+1}...")
    prep_utils.empty_directory(train_directory) # just to make sure no training images get mixed on restart

    #setup env variables etc.
    prep_utils.setup_training_environment(train_directory, results_dir, training_model, training_method, jobs[i]['config']['name'])
    images = prep_utils.load_images(base_dir, jobs[i]['config']['name'])
    prep_utils.move_images(jobs[i]['config']['name'], images, train_directory) # unnötig, da hier nur die configs erstellt werden

    #create and move training_parameter files into results folder
    config_save_path = os.path.join(os.environ.get("OUTPUT_DIR"), f"{job['config']['name']}.yaml")
    with open(config_save_path, "w") as f:
      yaml.dump(job, f, default_flow_style=False)

    run_job(job)

    prep_utils.empty_directory(train_directory)

Running job 1...
Directory does not exist: /content/train_images
Saved /content/train_images/hannah_1.jpg
Saved /content/train_images/hannah_2.jpg
Saved /content/train_images/hannah_3.jpg
Saved /content/train_images/hannah_4.jpg
Saved /content/train_images/hannah_5.jpg
Saved /content/train_images/hannah_6.jpg
Saved /content/train_images/hannah_7.jpg
Saved /content/train_images/hannah_8.jpg
Saved /content/train_images/hannah_9.jpg
Saved /content/train_images/hannah_10.jpg


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecate

{
    "type": "sd_trainer",
    "training_folder": "/content/drive/My Drive/results/lora/flux/hannah",
    "device": "cuda:0",
    "trigger_word": "ohwx",
    "network": {
        "type": "lora",
        "linear": 16,
        "linear_alpha": 16
    },
    "save": {
        "dtype": "float16",
        "save_every": 200,
        "max_step_saves_to_keep": 4
    },
    "datasets": [
        {
            "folder_path": "/content/train_images",
            "caption_ext": "txt",
            "caption_dropout_rate": 0.05,
            "shuffle_tokens": false,
            "cache_latents_to_disk": true,
            "resolution": [
                512,
                768,
                1024
            ]
        }
    ],
    "train": {
        "batch_size": 1,
        "steps": 3000,
        "gradient_accumulation_steps": 1,
        "train_unet": true,
        "train_text_encoder": false,
        "content_or_style": "balanced",
        "gradient_checkpointing": true,
        "noise_scheduler": "

/content/ai-toolkit/extensions_built_in/sd_trainer/SDTrainer.py:61: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


transformer/config.json:   0%|          | 0.00/378 [00:00<?, ?B/s]

...

Saving at step 2800


nils:  93%|█████████▎| 2799/3000 [1:47:02<07:52,  2.35s/it, lr: 1.0e-04 loss: 4.335e-01]

Saved to /content/drive/My Drive/results/lora/flux/nils/nils/optimizer.pt
Removing old save: /content/drive/My Drive/results/lora/flux/nils/nils/nils_000002000.safetensors


nils: 100%|█████████▉| 2999/3000 [1:54:38<00:02,  2.29s/it, lr: 1.0e-04 loss: 4.141e-01]



Saved to /content/drive/My Drive/results/lora/flux/nils/nils/optimizer.pt
Removed /content/train_images/nils_4.jpg
Skipped non-file: /content/train_images/_latent_cache
Removed /content/train_images/nils_5.jpg
Removed /content/train_images/.aitk_size.json
Removed /content/train_images/nils_9.jpg
Removed /content/train_images/nils_1.jpg
Removed /content/train_images/nils_3.jpg
Removed /content/train_images/nils_2.jpg
Removed /content/train_images/nils_10.jpg
Removed /content/train_images/nils_7.jpg
Removed /content/train_images/nils_8.jpg
Removed /content/train_images/nils_6.jpg
Directory /content/train_images has been emptied.


Inference prep (auslagern?) siehe inference notebook

In [ ]:
# from diffusers import AutoPipelineForText2Image
# import torch
# pipeline = AutoPipelineForText2Image.from_pretrained(
#     "black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16
# ).to("cuda")
# pipeline.load_lora_weights("sayakpaul/yarn_art_lora_flux", weight_name="pytorch_lora_weights.safetensors")
# image = pipeline("a puppy in a pond, yarn art style", guidance_scale=3.5, height=768).images[0]
# image.save("yarn.png")